In [7]:
import csv
from datetime import datetime
import numpy as np
import pandas as pd
import time 
from datetime import datetime
import os
from os import path
from glob import glob

import ijson
from tqdm import tqdm

## Import JSON 

In [8]:
zip_dir  = "../data/zip2"

files = glob(os.path.join(zip_dir, '*'))
for filename in files:
    if '.zip' in filename:
        pass
    elif '.json' in filename:
        pass
    else:
        newfile = filename+'.json'
        os.rename(filename, newfile)

jsons = glob(os.path.join(zip_dir, '*.json'))

In [9]:
# jsons = jsons[:2]
jsons

['../data/zip2/part-v003-o001-r-00000.json',
 '../data/zip2/part-v003-o001-r-00001.json',
 '../data/zip2/part-v003-o001-r-00002.json',
 '../data/zip2/part-v003-o001-r-00003.json',
 '../data/zip2/part-v003-o001-r-00004.json',
 '../data/zip2/part-v003-o001-r-00005.json',
 '../data/zip2/part-v003-o001-r-00006.json',
 '../data/zip2/part-v003-o001-r-00007.json',
 '../data/zip2/part-v003-o001-r-00008.json',
 '../data/zip2/part-v003-o001-r-00009.json']

## Add location data

In [10]:
# jsons = ['../data/part_v003_o001_r_00000.json', ..., '../data/part_v003_o001_r_00001.json']

dfrts = []
dfors = []
for i, json in enumerate(jsons):
    # Instance Preparation
    dates = []
    tweets = []
    user_ids = []
#     rts = []

#     # tweet's location
#     place_tweet = []
    # tweet's location
    place_fullname_tweet = []
    # tweet's country
    nationality = []
    
    start = time.time()
    # Load json file: date and tweet
    with open(json, 'r', encoding='utf8') as file:
        pet_parse = ijson.parse(file, multiple_values=True)
        for prefix, event, value  in pet_parse:
            # Date
            if prefix == 'created_at':
                dates.append(datetime.strptime(value, '%a %b %d %H:%M:%S +0000 %Y'))
            # Tweet 
            if prefix == 'text':
                tweets.append(value.replace('\n', '').replace('\t', '').replace('\r', '').replace('\r\n', '').replace('　', '')) # Delte space and indet and \r
            # User id 
            if prefix == 'user.id':
                user_ids.append(value)
                
#             # RT Flag
#             if len(dates)-1 == len(rts) and prefix == 'retweeted_status':
#                 rts.append(True)
#             if len(dates)-2 == len(rts):
#                 rts.append(False)
            
#             # tweet's location
#             if prefix == 'place.name':
#                 place_tweet.append(value)
#             if len(dates)-2 == len(place_tweet):
#                 place_tweet.append('nan')

            # tweet's location - fullname
            if prefix == 'place.full_name':
                place_fullname_tweet.append(value)
            if len(dates)-2 == len(place_fullname_tweet):
                place_fullname_tweet.append('nan')
                
            # tweet's nationality
            if prefix == 'place.country':
                nationality.append(value)
            if len(dates)-2 == len(nationality):
                nationality.append('nan')

        # Add missing data
#         if len(dates) != len(rts):
#                 rts.append(False)
        if len(dates) != len(place_fullname_tweet):
                place_fullname_tweet.append('nan')
        if len(dates) != len(nationality):
                nationality.append('nan')

    print("Loading json elapsed_time:{0}".format(time.time() - start) + "[sec]")
    
#     print(len(dates), len(tweets), len(user_ids), len(rts), len(nationality), len(place_tweet))
    
    # Create dataframe
    data = np.vstack([user_ids, dates, tweets, nationality, place_fullname_tweet]).T
    df = pd.DataFrame(data, columns=['user_id', 'date', 'tweet', 'nationality', 'place_tweet'])
    
    import datetime as dt
    
    # Df only with location data
    df = df[df.place_tweet != 'nan']
    
    # Change time zone
    df['date'] = df['date'] + dt.timedelta(hours = 9)
    
    # Date
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hour'] = df['date'].dt.hour
    df['time'] = df['date'].dt.time
    df['date'] = df['date'].dt.date
    
#     # divide df by RT_flag 
#     dfrt = df[df['RT_flag'] == True]
#     dfor = df[df['RT_flag'] == False]
    
    # Save to csv file
    df.to_csv('../data/dfs_location2/df_lct'+str(i)+'.csv')
#     dfrt.to_csv('../data/dfs_location/dfrt_lct'+str(i)+'.csv')
    
    # Append to dfs
#     dfrts.append(dfrt)
#     dfors.append(dfor)

Loading json elapsed_time:718.873034954071[sec]
Loading json elapsed_time:648.138466835022[sec]
Loading json elapsed_time:526.3302919864655[sec]
Loading json elapsed_time:507.3329060077667[sec]
Loading json elapsed_time:496.4747657775879[sec]
Loading json elapsed_time:536.7752206325531[sec]
Loading json elapsed_time:555.0317189693451[sec]
Loading json elapsed_time:566.3812553882599[sec]
Loading json elapsed_time:529.7259113788605[sec]
Loading json elapsed_time:740.5443599224091[sec]


In [107]:
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 10000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 


## Assign area 

In [224]:
area = pd.read_csv('../data/area_japan.csv')
t = pd.read_csv('../data/dfs_location/df_lct1.csv').drop('Unnamed: 0', axis=1)
area_jp = area.drop(['pref_en'], axis=1).set_index(['pref_jp']).Area.to_dict()
area_en = area.drop(['pref_jp'], axis=1).set_index(['pref_en']).Area.to_dict()

In [225]:
area = []
for natio, place in zip(t.nationality, t.place_tweet):
    if natio == '日本' and place.split(' ')[0] in area_jp:
        area.append(area_jp[place.split(' ')[0]])
    elif natio == '日本' and place.split(' ')[-1] in area_jp:
        area.append(area_jp[place.split(' ')[-1]])
    elif natio == 'Japan' and place.split(', ')[-1] in area_en:
        area.append(area_en[place.split(', ')[-1]])
    elif natio == 'Japan' and place.split(', ')[0] in area_en:
        area.append(area_en[place.split(', ')[0]])
    else:
        area.append('nan')

In [226]:
t['area'] = area

In [227]:
t

,user_id,date,tweet,nationality,place_tweet,year,month,day,hour,time,area
0,823461826339872768,2020-02-27,トルちゃん最終日、EXジム夏の思い出、自色で金ジム、終わりギリ大親友1人と人が来なくて困った...,日本,宮城 青葉区,2020,2,27,2,02:14:36,東北地方
1,200059773,2020-02-27,まだ前半だけどシナリオ的には天司たちの物語と匹敵するほどではないって感じw pvは壮大すぎで...,中华人民共和国,"广东, 中华人民共和国",2020,2,27,2,02:08:45,nan
2,1221438530867515392,2020-02-27,追加B'zhide←バンドじゃなくてもでも絵を描く時は絶対#米津玄師固定されました。最近描い...,Japan,"Minamiboso-shi, Chiba",2020,2,27,2,02:08:25,関東地方
3,140297465,2020-02-27,どえらいエロニキビが治らない。困った。,日本,鹿児 鹿児島市,2020,2,27,2,02:05:23,九州・沖縄地方
4,931291856477986816,2020-02-27,おはようございます。って言うかこんばんわ。m(_ _)m昨日で熊本での仕事も終わり、さ〜呑み...,日本,熊本 宇城市,2020,2,27,2,02:02:48,九州・沖縄地方
...,...,...,...,...,...,...,...,...,...,...,...
4659,1102038905811955713,2020-02-24,@kento47011659 原因分からないのが不安ですよね？病院で検査してくださいね。,Japan,"Numazu-shi, Shizuoka",2020,2,24,5,05:02:55,中部地方
4660,4799944992,2020-02-24,何曲か歌った所で「名古屋の名前を出して大阪で煽ったんだ今日盛り上がらなかったら7曲で帰る❗・...,日本,岐阜 輪之内町,2020,2,24,4,04:56:59,中部地方
4661,392696345,2020-02-24,ウォルトディズニーもスティーブ・ジョブズも好きなんやけど、2人は亡くなってる。今では偉人だの...,France,"Paris, France",2020,2,24,4,04:50:57,nan
4662,2220016902,2020-02-24,宇野から高松に四国フェリーと同じくらいの値段と時刻で行く方法みつけた直島経由して高松に行くの...,日本,道の駅 みやま公園,2020,2,24,4,04:49:30,nan


In [222]:
n = t[t.area == 'nan']
n2 = n[n.nationality == '日本']

In [223]:
n2

,user_id,date,tweet,nationality,place_tweet,year,month,day,hour,time,area
10,214091568,2020-02-27,今日はお誕生日のこちらの方の落語会へー！！おめでとうございますー㊗️今日も笑わせてもらいまし...,日本,あべのハルカス (Abeno Harukas),2020,2,27,1,01:52:29,nan
217,283672965,2020-02-26,先週は、シゲ演出の『サゼン幕』を観てきました✨後味の良い舞台は大好きです‼️号泣を懸念してタ...,日本,あうるすぽっと (舞台芸術交流センター),2020,2,26,22,22:47:25,nan
224,1020450848395046912,2020-02-26,【新宿SAMURAIありがとう】💫2/26meiyoセットリスト💫1.シトラス2.溶ける青春...,日本,Shinjuku SAMURAI,2020,2,26,22,22:44:14,nan
264,1481572735,2020-02-26,マジかぁやっぱそうなるよなぁめっちゃ楽しみにしてたのに...(◞‸◟ㆀ)でもメンバーの方がガ...,日本,バスタ新宿 (Shinjuku Expressway Bus Terminal),2020,2,26,22,22:22:50,nan
268,184733957,2020-02-26,お腹いっぱいになって落ち着いたのでデザートと紅茶を頂きながら東京ドームを眺めて想いにふける。...,日本,叙々苑,2020,2,26,22,22:19:19,nan
...,...,...,...,...,...,...,...,...,...,...,...
4596,786091920145735680,2020-02-24,旅行記録1日目。京都水族館を満喫。梅小路公園で意味もなく子どもたちと鬼ごっこして疲れる。ホテ...,日本,京都水族館 (KYOTO AQUARIUM),2020,2,24,7,07:20:27,nan
4606,78149438,2020-02-24,かわいいげんき😍#いわてS1スイーツフェア https://t.co/bOOXYHfVDG,日本,ビッグルーフ滝沢,2020,2,24,7,07:07:52,nan
4627,1085457911332036609,2020-02-24,私は喧嘩した相手をブロックまでしなかった。たぶんこれもその人は見るだろう。今はFFではないそ...,日本,日本,2020,2,24,6,06:34:42,nan
4628,1085457911332036609,2020-02-24,朝はや！薬入れたからもうちょっと寝るけど、これ見るかわからんけど、Twitterっていろんな...,日本,日本,2020,2,24,6,06:34:41,nan


In [186]:
a= t.place_tweet[0].split(' ')

In [193]:
t[t.nationality == 'Japan']

,user_id,date,tweet,nationality,place_tweet,year,month,day,hour,time,area
2,1221438530867515392,2020-02-27,追加B'zhide←バンドじゃなくてもでも絵を描く時は絶対#米津玄師固定されました。最近描い...,Japan,"Minamiboso-shi, Chiba",2020,2,27,2,02:08:25,関東地方
16,1217417832020922368,2020-02-27,そんな中で、私が学んだ空手は他の空手流派には無い「サバキ」という技術がありました。これは打撃...,Japan,"Itabashi-ku, Tokyo",2020,2,27,1,01:41:40,関東地方
19,1221438530867515392,2020-02-27,@95nEcyc70XvOl3i ビデオ、写真、山程撮ってます☆そして既に懐かしい😁💦嫌々期...,Japan,"Minamiboso-shi, Chiba",2020,2,27,1,01:34:32,関東地方
20,1217417832020922368,2020-02-27,皆さんこれ、どう思います？ケンカって、顔面パンチするでしょ？掴むでしょ？ケンカに反則も何もな...,Japan,"Itabashi-ku, Tokyo",2020,2,27,1,01:34:19,関東地方
27,1217417832020922368,2020-02-27,唐突ですが、ケンカの話をします。空手を学ぶと、ケンカに弱くなります。黒帯まで飛び抜けてしまえ...,Japan,"Itabashi-ku, Tokyo",2020,2,27,1,01:19:58,関東地方
...,...,...,...,...,...,...,...,...,...,...,...
4626,999942880486608897,2020-02-24,今日のパピコ高熱を出したので病院に連れていく。待合室でしんどそうだったので肩にもたれさせてや...,Japan,"Sapporo-shi Kita-ku, Hokkaido",2020,2,24,6,06:39:40,北海道地方
4631,984416076220919808,2020-02-24,@7eGKVE51tuxuPyg 糸井の守備は不安だらけやから高山使うべきだと思います,Japan,"Ikaruga-cho, Nara",2020,2,24,6,06:29:38,近畿地方
4649,1221774819248594945,2020-02-24,@Sugi103258 @gorillaman017 お金がないので困りました,Japan,"Takamatsu-shi, Kagawa",2020,2,24,5,05:34:11,四国地方
4653,1227625153397309442,2020-02-24,独居生活の独居老人の孤独さは悲惨過ぎるものだとつくづく思った。家族が一人も居ない独居老人はこ...,Japan,"Higashisonogi-cho, Nagasaki",2020,2,24,5,05:13:47,九州・沖縄地方


## Read csv

In [17]:
# Read ORs
zip_dir  = "../data/dfs" # new_dfs2_jst
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfors = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    dfors.append(df)

In [7]:
# Read Rts
zip_dir  = "../data/new_dfs2_jst"
csvfiles = glob(os.path.join(zip_dir, 'dfor*'))

dfors = []
for csvfile in csvfiles:
    df1 = pd.read_csv(csvfile).drop('Unnamed: 0', axis=1)
    dfors.append(df1)

## Check csv data

In [5]:
for df in dfors:
    print(df.RT_flag.unique())

[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]
[False]


In [5]:
for df in dfrts:
    print(df.RT_flag.unique())

[True]
[True]
[True]
[True]
[True]
[True]
[True]
[True]
[True]
[True]


## df info

In [7]:
c = [len(df) for df in dfors]
print('The amount of tweets included RT:', sum(c))

The amount of tweets included RT: 9286476


In [34]:
c = [len(df) for df in dfrts]
print('The amount of tweets included RT:', sum(c))

The amount of tweets included RT: 6692071


## Create csv by day

In [65]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_day(dfors, filetype, savedir, key_lang):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')
    
    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'
    
    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
            
    # Find tweets
    for dfor in tqdm(dfors):
         # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        # Add columns of keywords whose cell have 1 if this tweet includes a keyword 
        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        # Make rows
        uni_dates = dfor_c['date'].tolist()
        uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
        rows = []
        for date in uni_dates:
            d = {}
            d['date'] = str(date)
            for col in filenames:
                _df = dfor_c.groupby('date').get_group(date)
                d[col] = _df[col].sum()
            rows.append(d)
        
        # Make cols
        cols = filenames.copy()
        cols.insert(0, 'date')
        
        # Make dfs with rows and cols
        dft = pd.DataFrame(columns=cols)
        for row in rows:
            dft = dft.append(row, ignore_index=True) 
        dfs.append(dft)
        
    # Finally Connect dfs
    for i, df in enumerate(dfs):
        if i == 0:
            dfnew = df
        else:
            dfnew = pd.concat([dfnew, df], axis=0)
            
    # Groupby and sort by date
    dfnew = dfnew.groupby('date').sum()
    
    # Save
    outname = filetype+'_original.csv'
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)
    savename = os.path.join(outdir, outname)
    dfnew.to_csv(savename)

    return dfnew


## Run day

In [66]:
# OR
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    dfnew = findtwt_day(dfors, filetype=filetype, savedir='../new_results_jst/orjp/day', key_lang='jp') # new_results_jst2

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [01:29<00:00,  8.92s/it]


In [81]:
# OR
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    dfnew = findtwt_day(dfors, filetype=filetype, savedir='../new_results_jst/oren/day', key_lang='en')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:13<00:00,  1.31s/it]


In [11]:
# RT
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    dfnew = findtwt_day(dfrts, filetype=filetype, savedir='../new_results_jst/rtjp/day', key_lang='jp')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [03:53<00:00, 23.33s/it]


In [67]:
# RT
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    dfnew = findtwt_day(dfrts, filetype=filetype, savedir='../new_results_jst/rten/day', key_lang='en')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:17<00:00,  1.74s/it]


## Create csv by hour 

In [10]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_hour(dfors, filetype, savedir, key_lang):
    # Prepartion of dfw 
    results = []
    dfs = []
    dfw = pd.read_csv('../data/WordTimeSeries.csv', encoding='utf-8').rename(columns={'Unnamed: 0': 'types',
                                                                                    'file name': 'file_name', 
                                                                                    'orignal form': 'orignal_form',
                                                                                    'English translation': 'English_translation'})
    booleanDictionary = {True: 'TRUE', False: 'FALSE'} # Drop nan and False in order to use query 
    dfw = dfw.replace(booleanDictionary)
    dfw = dfw.dropna(how='all')

    # Prepration of keywords
    filenames = [filename for filename in dfw.file_name.tolist() if filetype in filename]
    xd = {}
    xd['T'] = 'file_name.str.contains("T")'
    xd['D'] = 'file_name.str.contains("D")'
    xd['A'] = 'file_name.str.contains("A")'
    xd['V'] = 'file_name.str.contains("V")'
    xd['F'] = 'file_name.str.contains("F")'
    xd['C'] = 'file_name.str.contains("C")'

    query = xd[filetype]
    dfw_c = dfw.query(query, engine='python')
    
    # Keyword 
    if key_lang == 'jp':
        keywords = dfw_c.orignal_form.tolist() # English.ver: keywords = dfw_c.English_translation.tolist() 
    elif key_lang == 'en':
        keywords = dfw_c.English_translation.tolist() 
    else:
        print('Put accurate keyword language')
    
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]

        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        dfs.append(dfor_c)
        
    # Create csv file by day
    all_df = []
    nextdf, nextdf_date = '', ''
    
    # Save dir
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)

    for i, df in enumerate(dfs):
        unique_dates = df['date'].tolist()
        unique_dates = sorted(set(unique_dates), key=unique_dates.index) # date values
        df_date = df.groupby('date') # df grouped by date   

        for date in unique_dates:
            df_oneday = df_date.get_group(date) # df of one day
            dfh = df_oneday.groupby('hour').sum()
            dfh = dfh.drop(['year', 'month', 'user_id', 'RT_flag', 'day'],axis=1)
            
            # Save csv
            outname = filetype+str(date)+'.csv'
            savename = os.path.join(outdir, outname)
            
            if os.path.exists(savename):
                # File1
                existedfile = pd.read_csv(savename)
                # いったんcsvにする
                _saveonce = 'once.csv'
                dfh.to_csv(_saveonce)
                # File2 
                once = pd.read_csv(_saveonce)
                # Concat 
                df_oneday1 = pd.concat([existedfile, once], axis=0)
                # Finally save
                dfh = df_oneday1.groupby('hour').sum()
                
            # Save to csv
            dfh.to_csv(savename)
          

## Run Orjp, Oren, Rtjp, Rten

In [11]:
# Orjp
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfors, filetype=filetype, savedir='../new_results_jst2/orjp/hour', key_lang='jp')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [01:31<00:00,  9.11s/it]


In [88]:
# Oren
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfors, filetype=filetype, savedir='../new_results_jst2/oren/hour', key_lang='en')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:12<00:00,  1.30s/it]


In [100]:
# Rtjp
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfrts, filetype=filetype, savedir='../new_results_jst2/rtjp/hour', key_lang='jp')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [01:56<00:00, 11.65s/it]


In [101]:
# Rten
filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
# filetypes = ['D', 'A', 'V', 'F', 'C']
for filetype in filetypes:
    findtwt_hour(dfrts, filetype=filetype, savedir='../new_results_jst2/rten/hour', key_lang='en')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:17<00:00,  1.75s/it]


## Check 

In [77]:
dft = pd.read_csv('../results/rtjp/hour/T2020-02-28.csv')
dft.T1.sum()

7827

In [ ]:
	T1
21	2958
22	2637
23	2546
24	4883
25	3352
26	2553
27	3206
28	5117
29	7827
30	2925

## コロナ含有率

In [50]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwt_corona(dfors, filetype="T"):
    # Prepration of keywords
    ## コロナ専用
    keywords = ['コロナ']
    
    c = [] # count volume
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        c.append(len(dfor_c))
    
    print(sum(c), 'tweets conatin "コロナ"')

# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
findtwt_corona(dfrts, filetype="T")
          

100%|██████████| 10/10 [00:04<00:00,  2.15it/s]

963821 tweets conatin "コロナ"


## Corona by day

In [102]:
# filetypes = ['T', 'D', 'A', 'V', 'F', 'C']
def findtwtCrn_day(dfors, savename, savedir):
    
    dfs = []

    # Keyowrds
    keywords = ['コロナ', 'Corona', 'corona']
    filenames = ['コロナ', 'Corona', 'corona']

    
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet include keywords
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]
        
        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        # Make rows
        uni_dates = dfor_c['date'].tolist()
        uni_dates = sorted(set(uni_dates), key=uni_dates.index) # date values
        rows = []
        for date in uni_dates:
            d = {}
            d['date'] = str(date)
            for col in filenames:
                _df = dfor_c.groupby('date').get_group(date)
                d[col] = _df[col].sum()
            rows.append(d)
        
        # Make cols
        cols = filenames.copy()
        cols.insert(0, 'date')
        
        # Make dfs with rows and cols
        dft = pd.DataFrame(columns=cols)
        for row in rows:
            dft = dft.append(row, ignore_index=True)
        
        dfs.append(dft)

    # Finally Connect dfs
    for i, df in enumerate(dfs):
        if i == 0:
            dfnew = df
        else:
            dfnew = pd.concat([dfnew, df], axis=0)
            
    # Groupby and sort by date
    dfnew = dfnew.groupby('date').sum()
    
    # Save
    outname = savename+'.csv'
    outdir = savedir

    if not os.path.exists(outdir):
            os.makedirs(outdir)

    savename = os.path.join(outdir, outname)    
    dfnew.to_csv(savename)

    return dfnew

In [94]:
dfnew = findtwtCrn_day(dfors, savename='corona', savedir='../new_results_jst/Corona/or')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:18<00:00,  1.89s/it]


In [103]:
dfnew = findtwtCrn_day(dfrts, savename='corona', savedir='../new_results_jst2/Corona/rt')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


## Corona by hour

In [91]:
def findtwtCrn_hour(dfors, savename, savedir):
    # Prepartion 
    results = []
    dfs = []
    
    # Save dir
    outdir = savedir
    if not os.path.exists(outdir):
            os.makedirs(outdir)
    
    # Keyowrds
    keywords = ['コロナ', 'Corona', 'corona']
    filenames = ['コロナ', 'Corona', 'corona']
    
    # Find tweets
    for dfor in tqdm(dfors):
        # Extract tweet
        dfor_c = dfor[dfor['tweet'].str.contains('|'.join(keywords))]

        for col, key in zip(filenames, keywords):
            l = []
            for row in dfor_c.itertuples():
                if key in row.tweet:
                    l.append(1)
                else:
                    l.append(0)
            dfor_c[col] = l
        
        dfs.append(dfor_c)
        
    # Create csv file by day
    all_df = []
    nextdf, nextdf_date = '', ''
    
    # Save dir
    outdir = savedir
    filetype = savename
    if not os.path.exists(outdir):
            os.makedirs(outdir)

    for i, df in enumerate(dfs):
        unique_dates = df['date'].tolist()
        unique_dates = sorted(set(unique_dates), key=unique_dates.index) # date values
        df_date = df.groupby('date') # df grouped by date   
        
        for date in unique_dates:
            df_oneday = df_date.get_group(date) # df of one day
            dfh = df_oneday.groupby('hour').sum()
            dfh = dfh.drop(['year', 'month', 'user_id', 'RT_flag', 'day'],axis=1)
            
            outname = filetype+str(date)+'.csv'
            savename = os.path.join(outdir, outname)
            
            if os.path.exists(savename):
                # File1
                existedfile = pd.read_csv(savename)
                
                # いったんcsvにする
                _saveonce = 'once.csv'
                dfh.to_csv(_saveonce)
                # File2 
                once = pd.read_csv(_saveonce)
                
                # Concat 
                df_oneday1 = pd.concat([existedfile, once], axis=0)

                # Finally save
                dfh = df_oneday1.groupby('hour').sum()
                
            # Save to csv
            dfh.to_csv(savename)
          

In [92]:
findtwtCrn_hour(dfors, savename='corona', savedir='../new_results_jst2/Corona/or')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:09<00:00,  1.02it/s]


In [104]:
findtwtCrn_hour(dfrts, savename='corona', savedir='../new_results_jst/Corona/rt')

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 10/10 [00:15<00:00,  1.54s/it]


In [115]:
t1 = pd.read_csv('../new_results_jst/rten/hour/V2020-02-01.csv')
t2 = pd.read_csv('../new_results/rten/hour/V2020-01-31.csv')


In [117]:
print(t1.loc[0:8,'V4'].sum(), t2.V4.sum())

0 0


In [114]:
dfrts[0].head()

,user_id,date,tweet,RT_flag,year,month,day,hour,time
0,1111855071380168705,2020-03-31,RT @k7LssPYI5D85fxI: https://t.co/5KgwBN00NS志村...,True,2020,3,31,23,23:59:58
1,127471475,2020-03-31,RT @84MadokaMary: この国には知らんだけでめちゃめちゃな量の社会保障があるの...,True,2020,3,31,23,23:59:58
2,138821520,2020-03-31,RT @zibumitunari: 明日はエイプリルフールだが、コロナ関連の嘘は絶対にやめよ...,True,2020,3,31,23,23:59:58
3,962192359059472384,2020-03-31,RT @akagiichirou: マスクの７割が中国からの輸入に頼ってきたが流通網が混乱す...,True,2020,3,31,23,23:59:58
4,128360620,2020-03-31,RT @bbcnewsjapan: BBCニュース - ハンガリー政府、新型ウイルス対策で強...,True,2020,3,31,23,23:59:57
